In [1]:
import pandas as pd
import glob
import numpy as np
import scipy.sparse as ss
from collections import Counter
import time
#import pyranges as pr
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,collect_list, concat, lit, when, monotonically_increasing_id, concat_ws, broadcast
import pyspark.sql.functions as F
# load in all files at the top, assert all files are correct length
# make their name the same as the input arg, minuus 'input'
spark = SparkSession.builder.master("local").appName("parquet").getOrCreate()

22/07/25 19:40:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
kutd_cadd_scored_files = glob.glob('/sbgenomics/project-files/KUTD_GF_*')

assert len(kutd_cadd_scored_files) == 44

In [3]:
%%time
li = []

# may need to use dask here, the data from each person is going to be bigger bc of +1 to start position
for n,filename in enumerate(kutd_cadd_scored_files):
    print(n,filename.split('/')[-1])
    #df = pd.read_parquet(filename)
    df = spark.read.parquet(filename)
    print(df.rdd.getNumPartitions())
    #li.append(df)
    
    if n == 0:
        frame = df
    else:
        frame = frame.union(df)

#frame = pd.concat(li, axis=0, ignore_index=True)

# join this frame w/ the main cadd snp file chrom by chrom -- possibly parallelize 
# then combine with the other variants using an app. this way we only need to load the large cadd snp file one time.

# get gene names from VWB consequences table

# then search for all of these variants (the unique set) in each gvcf/parquet file,
# we need to know if its a 0,1,2 or -9. how to do this quickly?

0 KUTD_GF_0ZR9EDHK_scored_variants.parquet


2
1 KUTD_GF_1K2GTEAM_scored_variants.parquet
2
2 KUTD_GF_3R71VD9D_scored_variants.parquet
2
3 KUTD_GF_451FYZ8Q_scored_variants.parquet
2
4 KUTD_GF_594ECSE4_scored_variants.parquet
2
5 KUTD_GF_65HR1A6Y_scored_variants.parquet
2
6 KUTD_GF_6S7MYRJ7_scored_variants.parquet
2
7 KUTD_GF_747J6KJ6_scored_variants.parquet
2
8 KUTD_GF_7D33NJE4_scored_variants.parquet
2
9 KUTD_GF_82BRARSA_scored_variants.parquet
2
10 KUTD_GF_9MAMSN75_scored_variants.parquet
2
11 KUTD_GF_AYSP3BP3_scored_variants.parquet
2
12 KUTD_GF_BB8E5PMX_scored_variants.parquet
2
13 KUTD_GF_DB3NYN22_scored_variants.parquet
2
14 KUTD_GF_DND6FMS5_scored_variants.parquet
2
15 KUTD_GF_E38XQY01_scored_variants.parquet
2
16 KUTD_GF_EF66M6D4_scored_variants.parquet
2
17 KUTD_GF_FPEYBPFH_scored_variants.parquet
2
18 KUTD_GF_FVQ9GQC9_scored_variants.parquet
2
19 KUTD_GF_FYZH55PM_scored_variants.parquet
2
20 KUTD_GF_H32KXJHW_scored_variants.parquet
2
21 KUTD_GF_JMAEQX7J_scored_variants.parquet
2
22 KUTD_GF_MV88VAZ9_scored_variants.parqu

In [5]:
frame.count()

6480517

In [4]:
#frame['unique_location'] = frame['chromosome'] +':'+ frame['start'].astype(str) +':'+ frame['end'].astype(str)
frame = frame.withColumn('unique_location',concat(col("chromosome"),
                                lit(":"),col("start"),lit(":"),col("end")))

In [5]:
frame_dedup = frame.drop_duplicates(['unique_location'])
frame_dedup.count()

472005

In [6]:
frame_dedup.select('chromosome','start','end','unique_variant_id','eqtl_boolean').repartitionByRange(30, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("KUTD_all_unique_variants.parquet")

In [7]:
!mv KUTD_all_unique_variants.parquet/ /sbgenomics/output-files/

In [13]:
frame_dedup.where(col('eqtl_boolean') == 0).count()

101189

## Concatenate CHD files 

In [2]:
chd_cadd_scored_files = glob.glob('/sbgenomics/project-files/CHD_BS_*')
assert len(chd_cadd_scored_files) == 711

In [3]:
chd_cadd_scored_files[:3]

['/sbgenomics/project-files/CHD_BS_0302Y3N5_scored_variants.parquet',
 '/sbgenomics/project-files/CHD_BS_03BQSV43_scored_variants.parquet',
 '/sbgenomics/project-files/CHD_BS_04R6HY6V_scored_variants.parquet']

In [4]:
chd_cadd_scored_files[-3:]

['/sbgenomics/project-files/CHD_BS_ZXMZDGG6_scored_variants.parquet',
 '/sbgenomics/project-files/CHD_BS_ZYG3D3D7_scored_variants.parquet',
 '/sbgenomics/project-files/CHD_BS_ZZNSNDVQ_scored_variants.parquet']

In [8]:
%%time
li = []

# may need to use dask here, the data from each person is going to be bigger bc of +1 to start position
for n,filename in enumerate(chd_cadd_scored_files[600:]):
    df = spark.read.parquet(filename)
    df = df.withColumn('unique_location',concat(col("chromosome"),lit(":"),col("start"),lit(":"),col("end")))
    unique_vars = df.select('chromosome','start','end','unique_variant_id','unique_location','eqtl_boolean')   
    
    if n == 0: frame = unique_vars
    else: frame = frame.union(unique_vars)
        
    if (n -1)% 10 == 0:
        print(n,filename.split('/')[-1])
        #print('DROPPING DUPLICATES')
        frame = frame.drop_duplicates(['unique_location'])

frame.drop_duplicates(['unique_location']).repartitionByRange(30, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("sixhundred_711_chd_unique_set.parquet")

1 CHD_BS_VR15YKJP_scored_variants.parquet
11 CHD_BS_VZ9W1E8S_scored_variants.parquet
21 CHD_BS_WDFYJ87C_scored_variants.parquet
31 CHD_BS_X1J6GHCV_scored_variants.parquet
41 CHD_BS_XDM3YN9Y_scored_variants.parquet
51 CHD_BS_XZ4834BN_scored_variants.parquet
61 CHD_BS_Y6N9XXAC_scored_variants.parquet
71 CHD_BS_YSY0Y7VA_scored_variants.parquet
81 CHD_BS_Z4PR01TV_scored_variants.parquet
91 CHD_BS_ZF22167H_scored_variants.parquet
101 CHD_BS_ZRK11YWW_scored_variants.parquet


CPU times: user 862 ms, sys: 264 ms, total: 1.13 s
Wall time: 23min 57s


In [7]:
len(chd_cadd_scored_files[600:])

111

In [5]:
!rm -r fivehundred_600_chd_unique_set.parquet/

In [10]:
a = spark.read.parquet('onehundred_200_chd_unique_set.parquet/')
b = spark.read.parquet('twohundred_300_chd_unique_set.parquet/')
c = spark.read.parquet('threehundred_400_chd_unique_set.parquet/')
d = spark.read.parquet('fourhundred_500_chd_unique_set.parquet/')
e = spark.read.parquet('fivehundred_600_chd_unique_set.parquet/')
f = spark.read.parquet('sixhundred_711_chd_unique_set.parquet/')

In [13]:
frame = a.union()

AttributeError: 'list' object has no attribute '_jdf'

In [14]:
from functools import reduce
from pyspark.sql import DataFrame

dfs = [a,b,c,d,e,f]
frame = reduce(DataFrame.unionAll, dfs)

In [16]:
frame_dedup = frame.drop_duplicates(['unique_location'])
frame_dedup.count()

1177569

In [17]:
frame_dedup.drop_duplicates(['unique_location']).repartitionByRange(30, "chromosome", "start").write\
      .mode('overwrite')\
      .partitionBy("chromosome")\
      .parquet("CHD_all_unique_variants.parquet")

In [18]:
frame_dedup.show(5)

+---------+---------+-------------------+--------------------+------------+----------+
|    start|      end|  unique_variant_id|     unique_location|eqtl_boolean|chromosome|
+---------+---------+-------------------+--------------------+------------+----------+
|100243666|100243669|10:100243666:CCTT:C|10:100243666:1002...|           1|        10|
|100256272|100256272|   10:100256272:C:A|10:100256272:1002...|           0|        10|
|100498711|100498711|   10:100498711:T:G|10:100498711:1004...|           0|        10|
|100741846|100741846|   10:100741846:A:C|10:100741846:1007...|           0|        10|
|100791306|100791306|   10:100791306:C:G|10:100791306:1007...|           1|        10|
+---------+---------+-------------------+--------------------+------------+----------+
only showing top 5 rows



In [19]:
frame_dedup.where(col('start')==col('end')).count()

1150778

In [21]:
!mv CHD_all_unique_variants.parquet/ /sbgenomics/output-files/